In [95]:
# DATA EXTRACTION


import requests, json
import pandas as pd

pd.set_option('display.max_columns', None)   # used to present desired number of columns

base_url = "https://api.openweathermap.org/data/2.5/weather?units=metric"
api_key = "7b755f3da019d2568444e285d46d2b54"
city = "Paris"
complete_url = base_url + "&appid=" + api_key + "&q=" + city

response = requests.get(complete_url)
data = response.json()   # converting response to readable to json form data
df = pd.json_normalize(data, sep='_') # normalize json data to tables to make it more readable
df
# data = pd.Dataframe(response)
# print(response.status_code)
# print(response.text)

# Error with data right now - Unstructured - contexts are not much clear

,weather,base,visibility,dt,timezone,id,name,cod,coord_lon,coord_lat,main_temp,main_feels_like,main_temp_min,main_temp_max,main_pressure,main_humidity,wind_speed,wind_deg,clouds_all,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset
0,"[{'id': 804, 'main': 'Clouds', 'description': ...",stations,10000,1700328596,3600,2988507,Paris,200,2.3488,48.8534,12.76,12.6,11.59,13.48,1016,96,5.66,220,100,2,2041230,FR,1700291021,1700323679


In [96]:
# DATA TRANSFORMATION


df.shape
df['weather']
df['weather'].to_json(orient='records')  # making data more readable

# Now Flatten this Data - using Pandas explode method

df_weather = pd.json_normalize(df['weather'].explode())
df_weather

# Removing icon column from Dataframe

df_weather.drop(columns = ['icon'], inplace=True)

# Now let's concatenate df_weather and df

df_weather.rename(columns = {'id': 'weather_id'}, inplace = True)
df_concat = pd.concat([df, df_weather], axis = 1)
df_concat

df_concat.drop(columns = ['weather'], inplace = True)

def convert_to_datetime(row, columns):
    dt = pd.to_datetime(row[columns], unit= 's')
    dt = dt + pd.to_timedelta(row['timezone'], unit='s')
    row[columns] = dt
    return row

df_concat = df_concat.apply(lambda x: convert_to_datetime(x, 'dt'), axis=1)


In [92]:
# DATA LOADING

# Download mysql server - install - create new schema
# SQL Alchemy
# Snowflake

from sqlalchemy import create_engine

username = 'data_etl'
password = 'pR3!aQ8sZ2wX'
database_name = 'weather_data'
host = 'local_host'
port = 3306

try:
    connection_string = f"mysql+mysql"

    engine = create_engine(connection_string)
    connection = engine.raw_connection()
    db_Info = connection.get_server_info()
    print('Connected', db_Info)

except Exception as e:
    print("Error", e)

df_concat.to_sql(name="city_data", con=engine, if_exists="append", index=False)